In [1]:
import requests
from lxml import etree
import json
import csv
import time
import pandas as pd

# 一、定义爬虫：
class FilmsSpider(object):
    def __init__(self):
        self.base_url = 'https://movie.douban.com/top250?start={}&filter='
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/73.0.3683.86 Chrome/73.0.3683.86 Safari/537.36'}
        self.data_of_films_dict = []


    # 1,构建所有的url：
    def get_url_list(self):
        url_list = []
        for i in range(0, 10):
            url = self.base_url.format(i*25)
            url_list.append(url)
        return url_list

    # 2,发请求：
    def send_request(self, url):
        data = requests.get(url, headers = self.headers).content.decode()
        return data

    # 3,解析数据：
    def parse_xpath_data(self, data):
        parse_data = etree.HTML(data)
        # 1,解析出所有的电影：
        films_list = parse_data.xpath('//ol[@class="grid_view"]/li')
        print(len(films_list))

        # 2,解析出电影的属性信息：
        for film in films_list:
            films_dict = {}
            # 1,排名：
            films_dict['film_ranking'] = film.xpath('.//em[@class=""]/text()')[0]  # .:表示当前路径, //：表示跨节点
            #print(film_ranking)

            # 2,电影名：
            films_dict['film_name'] = film.xpath('.//span[@class="title"]/text()')[0]
            #print(film_name)

            # 3,国家：
            films_dict['fiml_country'] = film.xpath('.//p[@class=""]/text()')[1][5:-6]
            #print(fiml_country)

            # 4,导演：
            films_dict['film_director'] = film.xpath('.//p[@class=""]/text()')[0][4:-6]
            #print(film_director)
            self.data_of_films_dict.append(films_dict)


    # 4,保存数据：
    def save_data(self):
        json.dump(self.data_of_films_dict, open('../OutPut/002,films_TOP250.json', 'w'))

    # 5,启动：
    def start(self):
        url_list = self.get_url_list()

        # 循环遍历发送请求：
        for url in url_list:
            print(url)
            data = self.send_request(url)
            self.parse_xpath_data(data)

        self.save_data()

In [2]:
# 二、运行爬虫：
start = time.time()

FilmsSpider().start()

end = time.time()
print('cost time: ', end-start, 's')

https://movie.douban.com/top250?start=0&filter=
25
https://movie.douban.com/top250?start=25&filter=
25
https://movie.douban.com/top250?start=50&filter=
25
https://movie.douban.com/top250?start=75&filter=
25
https://movie.douban.com/top250?start=100&filter=
25
https://movie.douban.com/top250?start=125&filter=
25
https://movie.douban.com/top250?start=150&filter=
25
https://movie.douban.com/top250?start=175&filter=
25
https://movie.douban.com/top250?start=200&filter=
25
https://movie.douban.com/top250?start=225&filter=
25
cost time:  4.877356290817261 s


In [3]:
# 三、将json数据转换成csv格式：列表数据转csv

# 1,读 创建文件：
json_fp = open('../OutPut/002,films_TOP250.json', 'r')
csv_fp = open('../OutPut/002,films_TOP250.csv', 'w')

# 2,提出表头 表内容：
data_list = json.load(json_fp)
sheet_title = data_list[0].keys()


sheet_data = []
for data in data_list:
    sheet_data.append(data.values())
# print(sheet_data)

# 3,csv写入器：
writer = csv.writer(csv_fp)

# 4,写入表头：
writer.writerow(sheet_title)

# 5,写入内容：
writer.writerows(sheet_data)

# 6,关闭两个文件：
json_fp.close()
csv_fp.close()

In [4]:
# 4,读取csv文件：
data = pd.read_csv('../OutPut/002,films_TOP250.csv')
data.head()

,film_ranking,film_name,fiml_country,film_director
0,1,肖申克的救赎,1994 / 美国 / 犯罪 剧情\n ...,导演: 弗兰克·德拉邦特 Frank Da...
1,2,霸王别姬,1993 / 中国大陆 香港 / 剧情 爱情...,导演: 陈凯歌 Kaige Chen ...
2,3,这个杀手不太冷,1994 / 法国 / 剧情 动作 犯罪\n...,导演: 吕克·贝松 Luc Besson ...
3,4,阿甘正传,1994 / 美国 / 剧情 爱情\n ...,导演: 罗伯特·泽米吉斯 Robert Z...
4,5,美丽人生,1997 / 意大利 / 剧情 喜剧 爱情 ...,导演: 罗伯托·贝尼尼 Roberto B...
